In [1]:
import pandas as pd
import re

# Dataset info

In [2]:
df = pd.read_csv('../data/raw/NLP training data 0816.csv')
df.drop_duplicates(inplace=True)
df.reset_index(inplace=True, )
df.head()

,index,question_id,client_id,campaign_id,question_category,question_subcategory,question_keyword,question_description
0,0,benefit_search-preventive_care-1,Aetna_Medicare,PDP_Conversion_SMS,Benefit Search,preventive care,Preventive care,Which plan(s) cover preventive benefits?
1,1,benefit_search-preventive_care-2,Aetna_Medicare,PDP_Conversion_SMS,Benefit Search,preventive care,Preventive care,How much does preventive benefits cost to me?
2,2,benefit_search-preventive_care-1,Aetna_Medicare,PDP_Conversion_SMS,Benefit Search,preventive care,Abdominal aortic aneurysm screening,Which plan(s) cover preventive benefits?
3,3,benefit_search-preventive_care-2,Aetna_Medicare,PDP_Conversion_SMS,Benefit Search,preventive care,Abdominal aortic aneurysm screening,How much does preventive benefits cost to me?
4,4,benefit_search-preventive_care-1,Aetna_Medicare,PDP_Conversion_SMS,Benefit Search,preventive care,Alcohol misuse screenings & counseling,Which plan(s) cover preventive benefits?


# Generation

## which plans

In [3]:
df.shape, df.question_description.unique().shape

((11434, 8), (284,))

In [4]:
mask = df.question_description.str.contains(']', regex=False)
df[mask]

,index,question_id,client_id,campaign_id,question_category,question_subcategory,question_keyword,question_description


In [5]:
mask = df.question_description.str.contains('Which plan', regex=False)
df[mask].shape, df[mask].sort_values(by='question_description').question_description.unique().shape

((8409, 8), (107,))

In [6]:
df[mask].question_description.unique()

array(['Which plan(s) cover preventive benefits?',
       'Which plan(s) cover outpatient hospital services?',
       'Which plan(s) cover ambulatory surgical center (ASC)?',
       'Which plan(s) cover outpatient hospital observation services?',
       'Which plan(s) cover outpatient breast procedure?',
       'Which plan(s) cover outpatient cardiovascular procedure?',
       'Which plan(s) cover outpatient digestive/gastrointestinal procedure?',
       'Which plan(s) cover outpatient eye procedure?',
       'Which plan(s) cover outpatient hematology procedure?',
       'Which plan(s) cover outpatient musculoskeletal procedure?',
       'Which plan(s) cover outpatient skin procedure?',
       'Which plan(s) cover outpatient vascular procedure?',
       'Which plan(s) cover outpatient dialysis procedure?',
       'Which plan(s) cover chemotherapy drugs?',
       'Which plan(s) cover durable medical equipment (e.g. wheelchair, oxygen)?',
       'Which plan(s) cover blood sugar meters?',

In [40]:
processed_df = df.copy()

def which_plan(line):
    
    
    if 'and what\'s the cost' in line['question_description']:
        line['question_description'] = re.sub(r"(?P<q>Which plan\(s\) cover) (?P<entity>.[^?]+)(?P<e> and what's the cost\?)", r'\g<q> [\g<entity>]\g<e>', line['question_description'])
    
    elif 'Which plan' in line['question_description']:
        if 'coverage?' not in line['question_description']:
            line['question_description'] = re.sub(r'(?P<q>cover|offer) (?P<entity>.[^?]+)', r'\g<q> [\g<entity>]', line['question_description'])  
        else:
            line['question_description'] = re.sub(r'(?P<q>cover|offer|have) (?P<entity>.[^?]+)(?P<e> coverage\?)', r'\g<q> [\g<entity>]\g<e>', line['question_description'])  
    
    elif 'How much does ' in line['question_description']:
        line['question_description'] = re.sub(r'(?P<q>(How much does)) (?P<entity>.[^?]+)(?P<e> cost to me\?)', r'\g<q> [\g<entity>]\g<e>', line['question_description'])
    
    elif 'What does ' in line['question_description']:
        line['question_description'] = re.sub(r'(?P<q>(What does)) (?P<entity>.[^?]+)(?P<e> mean\?)', r'\g<q> [\g<entity>]\g<e>', line['question_description'])
    
    elif 'What is the ' in line['question_description']:
        line['question_description'] = re.sub(r'(?P<q>(What is the)) (?P<entity>.[^?]+)', r'\g<q> [\g<entity>]', line['question_description'])
        
    elif 'I\'d like to talk to ' in line['question_description']:
        line['question_description'] = re.sub(r'(?P<q>(I\'d like to talk to)) (?P<entity>.[^?]+)', r'\g<q> [\g<entity>]', line['question_description'])
    
    return line
processed_df = processed_df.apply(which_plan, axis='columns')

Which plan(s) cover [my drug(s)] and what's the cost?
Which plan(s) cover [my drug(s)] and what's the cost?
Which plan(s) cover [my drug(s)] and what's the cost?
Which plan(s) cover [my drug(s)] and what's the cost?
Which plan(s) cover [my drug(s)] and what's the cost?
Which plan(s) cover [my drug(s)] and what's the cost?
Which plan(s) cover [my drug(s)] and what's the cost?
Which plan(s) cover [my drug(s)] and what's the cost?
Which plan(s) cover [my drug(s)] and what's the cost?
Which plan(s) cover [my drug(s)] and what's the cost?
Which plan(s) cover [my drug(s)] and what's the cost?
Which plan(s) cover [my drug(s)] and what's the cost?
Which plan(s) cover [my drug(s)] and what's the cost?
Which plan(s) cover [my drug(s)] and what's the cost?
Which plan(s) cover [my drug(s)] and what's the cost?
Which plan(s) cover [my drug(s)] and what's the cost?
Which plan(s) cover [my drug(s)] and what's the cost?
Which plan(s) cover [my drug(s)] and what's the cost?
Which plan(s) cover [my drug

In [29]:
mask = processed_df.question_description.str.contains('Which plan', regex=False)
processed_df[mask].question_description.unique()

array(['Which plan(s) cover [preventive benefits]?',
       'Which plan(s) cover [outpatient hospital services]?',
       'Which plan(s) cover [ambulatory surgical center (ASC)]?',
       'Which plan(s) cover [outpatient hospital observation services]?',
       'Which plan(s) cover [outpatient breast procedure]?',
       'Which plan(s) cover [outpatient cardiovascular procedure]?',
       'Which plan(s) cover [outpatient digestive/gastrointestinal procedure]?',
       'Which plan(s) cover [outpatient eye procedure]?',
       'Which plan(s) cover [outpatient hematology procedure]?',
       'Which plan(s) cover [outpatient musculoskeletal procedure]?',
       'Which plan(s) cover [outpatient skin procedure]?',
       'Which plan(s) cover [outpatient vascular procedure]?',
       'Which plan(s) cover [outpatient dialysis procedure]?',
       'Which plan(s) cover [chemotherapy drugs]?',
       'Which plan(s) cover [durable medical equipment (e.g. wheelchair, oxygen)]?',
       'Which plan(

In [30]:
mask = processed_df.question_description.str.contains('How much does ', regex=False)
processed_df[mask].question_description.unique()

array(['How much does [preventive benefits] cost to me?',
       'How much does [outpatient hospital services] cost to me?',
       'How much does [ambulatory surgical center (ASC)] cost to me?',
       'How much does [outpatient hospital observation services] cost to me?',
       'How much does [outpatient breast procedure] cost to me?',
       'How much does [outpatient cardiovascular procedure] cost to me?',
       'How much does [outpatient digestive/gastrointestinal procedure] cost to me?',
       'How much does [outpatient eye procedure] cost to me?',
       'How much does [outpatient hematology procedure] cost to me?',
       'How much does [outpatient musculoskeletal procedure] cost to me?',
       'How much does [outpatient skin procedure] cost to me?',
       'How much does [outpatient vascular procedure] cost to me?',
       'How much does [outpatient dialysis procedure] cost to me?',
       'How much does [chemotherapy drugs] cost to me?',
       'How much does [durable med

In [31]:
mask = ~processed_df.question_description.str.contains(']', regex=False)
processed_df[mask].question_description.unique()

array(["Which plan(s) cover my drug(s) and what's the cost?"],
      dtype=object)

# Merge keywords and phrases

In [20]:
processed_df.head()

,index,question_id,client_id,campaign_id,question_category,question_subcategory,question_keyword,question_description
0,0,benefit_search-preventive_care-1,Aetna_Medicare,PDP_Conversion_SMS,Benefit Search,preventive care,Preventive care,Which plan(s) cover [preventive benefits]?
1,1,benefit_search-preventive_care-2,Aetna_Medicare,PDP_Conversion_SMS,Benefit Search,preventive care,Preventive care,How much does [preventive benefits] cost to me?
2,2,benefit_search-preventive_care-1,Aetna_Medicare,PDP_Conversion_SMS,Benefit Search,preventive care,Abdominal aortic aneurysm screening,Which plan(s) cover [preventive benefits]?
3,3,benefit_search-preventive_care-2,Aetna_Medicare,PDP_Conversion_SMS,Benefit Search,preventive care,Abdominal aortic aneurysm screening,How much does [preventive benefits] cost to me?
4,4,benefit_search-preventive_care-1,Aetna_Medicare,PDP_Conversion_SMS,Benefit Search,preventive care,Alcohol misuse screenings & counseling,Which plan(s) cover [preventive benefits]?


In [26]:
merged_df = processed_df.copy()

def merge_data(line):
    if isinstance(line['question_keyword'], str):
        line['train_phase'] = re.sub(r'\[[^?]+\]', line['question_keyword'].lower(), line['question_description'])
    return line
merged_df = merged_df.apply(merge_data, axis='columns')
merged_df.head()

,campaign_id,client_id,index,question_category,question_description,question_id,question_keyword,question_subcategory,train_phase
0,PDP_Conversion_SMS,Aetna_Medicare,0,Benefit Search,Which plan(s) cover [preventive benefits]?,benefit_search-preventive_care-1,Preventive care,preventive care,Which plan(s) cover preventive care?
1,PDP_Conversion_SMS,Aetna_Medicare,1,Benefit Search,How much does [preventive benefits] cost to me?,benefit_search-preventive_care-2,Preventive care,preventive care,How much does preventive care cost to me?
2,PDP_Conversion_SMS,Aetna_Medicare,2,Benefit Search,Which plan(s) cover [preventive benefits]?,benefit_search-preventive_care-1,Abdominal aortic aneurysm screening,preventive care,Which plan(s) cover abdominal aortic aneurysm ...
3,PDP_Conversion_SMS,Aetna_Medicare,3,Benefit Search,How much does [preventive benefits] cost to me?,benefit_search-preventive_care-2,Abdominal aortic aneurysm screening,preventive care,How much does abdominal aortic aneurysm screen...
4,PDP_Conversion_SMS,Aetna_Medicare,4,Benefit Search,Which plan(s) cover [preventive benefits]?,benefit_search-preventive_care-1,Alcohol misuse screenings & counseling,preventive care,Which plan(s) cover alcohol misuse screenings ...


In [41]:
processed_df.question_description.unique()

array(['Which plan(s) cover [preventive benefits]?',
       'How much does [preventive benefits] cost to me?',
       'Which plan(s) cover [outpatient hospital services]?',
       'How much does [outpatient hospital services] cost to me?',
       'Which plan(s) cover [ambulatory surgical center (ASC)]?',
       'How much does [ambulatory surgical center (ASC)] cost to me?',
       'Which plan(s) cover [outpatient hospital observation services]?',
       'How much does [outpatient hospital observation services] cost to me?',
       'Which plan(s) cover [outpatient breast procedure]?',
       'How much does [outpatient breast procedure] cost to me?',
       'Which plan(s) cover [outpatient cardiovascular procedure]?',
       'How much does [outpatient cardiovascular procedure] cost to me?',
       'Which plan(s) cover [outpatient digestive/gastrointestinal procedure]?',
       'How much does [outpatient digestive/gastrointestinal procedure] cost to me?',
       'Which plan(s) cover [outp

In [39]:
processed_df.question_id.unique().shape

(296,)

# Live agent

In [46]:
df = pd.read_csv('../data/external/rasa_ask_human.csv')
df.head()

,phrase
0,Can I speak to anyone who can really help me?
1,Could I talk to [Tyrone King](name)?
2,I don't wanna talk to a bot
3,I dont like to talk to a machine
4,I want to talk to a human


In [ ]:
df.drop_duplicates(inplace=True)
df.to_csv('../data/external/rasa_ask_human_unique.csv', index=False)